# Hands -on example

In [2]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn

# The data

Wine quality dataset

In [3]:
data=pd.read_csv('wine+quality/winequality-red.csv',sep=';')
data.sample()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1258,6.8,0.64,0.0,2.7,0.123,15.0,33.0,0.99538,3.44,0.63,11.3,6


# Tracking Experiments

MLflow supports 2 types of backend stores : file store and database based store


In [4]:
#mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri="http://127.0.0.1:8000" # set to your server uri
mlflow.set_tracking_uri(uri="http://127.0.0.1:8000") # or set the MLFLOW_TRACKING_URI in the env

In [4]:
# mlflow.tracking.get_tracking_uri()


'http://127.0.0.1:8000'

In [11]:
mlflow.create_experiment('ElasticNet_wine')

'521025943128782150'

In [5]:
mlflow.set_experiment('ElasticNet_wine')

<Experiment: artifact_location='file:///C:/Users/aravind.raman/Downloads/mlflow_walkthrough/mlruns/634715942043803316', creation_time=1715706350593, experiment_id='634715942043803316', last_update_time=1715706350593, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [15]:
mlflow.start_run(experiment_id=)

# What do we track?
1.Code Version : Git commit hash used for the run

2.Start & End Time: Start and end time of the run

3.Source: WHat code ran?

4.Parameters: Key-value input parameters

5.Metrics: Key-value metrics, where the value is numeric (can be updated over the run)

6.Artifacts: Output files in any format.

In [15]:
def eval_metrics(actual,pred):
    # compute relevant metrics
    rmse=np.sqrt(mean_squared_error(actual,pred))
    mae=mean_absolute_error(actual,pred)
    r2=r2_score(actual,pred)
    return rmse,mae,r2
def load_data(data_path):
    data=pd.read_csv(data_path,sep=';')
    train,test=train_test_split(data)
    train_x=train.drop(['quality'],axis=1)
    test_x=test.drop(['quality'],axis=1)
    train_y=train[['quality']]
    test_y=test[['quality']]
    return train_x,train_y,test_x,test_y
def train (alpha=0.5 ,l1_ratio=0.5):
    warnings.filterwarnings('ignore')
    np.random.seed(42)
    data_path='wine+quality/winequality-red.csv'
    train_x,train_y,test_x,test_y=load_data(data_path)
    # Useful while running multiple models like Grid search CV
    with mlflow.start_run():
        lr=ElasticNet(alpha=alpha,l1_ratio=l1_ratio,random_state=42)
        lr.fit(train_x,train_y)
        predicted_qualities=lr.predict(test_x)
        (rmse,mae,r2)=eval_metrics(test_y,predicted_qualities)
        print(f'ElasticNet model alpha={alpha} and l1_ratio={l1_ratio}')
        print(f'RMSE: {rmse}')
        print(f'MAE: {mae}')
        print(f'R2: {r2}')

        mlflow.log_param(key='alpha',value=alpha)
        mlflow.log_param(key='l1_ratio',value=l1_ratio)
        mlflow.log_metric(key='rmse',value=rmse)
        mlflow.log_metrics({'mae':mae,'r2':r2})
        mlflow.log_artifact(data_path)
        print(f'Save to {mlflow.get_artifact_uri()}')
        mlflow.sklearn.log_model(lr,'model')
    

In [ ]:
# could also do
# with mlflow.start_run():
#     for epoch in range(0,3):
#         mlflow.log_metric(key="quality",value=2*epoch,step=epoch)
        

In [19]:
train(0.3,0.3)

ElasticNet model alpha=0.3 and l1_ratio=0.3
RMSE: 0.7036020488067385
MAE: 0.5643396524855138
R2: 0.1997076545227946
Save to mlflow-artifacts:/521025943128782150/d624741554aa4a889a3ce61672e6a617/artifacts


# 1.1 Comparing runs

Run mlflow ui in a terminal or http//127.0.0.0.1:5000 to view the experiment logs and visualise and compare different runs

# Packaging the experiment as a MLFlow project as conda env

Specify the entrypoint for this project by creating a MLproject file and adding an conda environment with a conda.yaml. You can copy the yaml file from the experiment logs.

To run this project, invoke mlflow run . -P alpha=0.42. After running this command, MLflow runs your training code in a new Conda environment with the dependencies specified in conda.yaml.

# Deploy the model

3. Deploy the model
Deploy the model locally by running

mlflow models serve -m mlflow-artifacts:/0/7f57b33eb7ba4fe194bc1a5a186428c5/artifacts/model/ -h 127.0.0.1 -p 8000

Test the endpoint:

curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://0.0.0.0:1234/invocations

You can also simply build a docker image from your model

mlflow models build-docker -m mlruns/1/d671f37a9c7f478989e67eb4ff4d1dac/artifacts/model/ -n elastic_net_wine

and run the container with

docker run -p 8080:8080 elastic_net_wine.

Or you can directly deploy to AWS sagemaker or Microsoft Azure ML.

In [2]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.search_experiments()
experiments = client.search_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='file:///C:/Users/aravind.raman/Downloads/mlflow_walkthrough/mlruns/634715942043803316', creation_time=1715706350593, experiment_id='634715942043803316', last_update_time=1715706350593, lifecycle_stage='active', name='ElasticNet_wine', tags={}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1715243610394, experiment_id='0', last_update_time=1715243610394, lifecycle_stage='active', name='Default', tags={}>]


In [3]:
# get the run
_run = client.get_run(run_id="7f57b33eb7ba4fe194bc1a5a186428c5")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.5722846717246248,
 'r2': 0.21978513651550247,
 'rmse': 0.7420620899060748}, params={'alpha': '0.42', 'l1_ratio': '0.1'}, tags={'mlflow.gitRepoURL': 'https://github.com/aravindmtiger/mlflow_walkthrough.git',
 'mlflow.log-model.history': '[{"run_id": "7f57b33eb7ba4fe194bc1a5a186428c5", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-05-15 07:14:27.693917", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"loader_module": "mlflow.sklearn", '
                             '"python_version": "3.6.8", "env": "conda.yaml"}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "0.21.3", '
                             '"serialization_format": "cloudpickle"}}, '
                             '"model_uuid": '
                             '"b030ade659

In [4]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)